In [0]:
from pyspark.sql.functions import current_timestamp, input_file_name


In [0]:
%sql
CREATE VOLUME If not EXISTS angad_kumar91.fraud_detection_raw_data_files.schema;


In [0]:
%sql

-- checkpoint parent volume
CREATE VOLUME IF NOT EXISTS angad_kumar91.fraud_detection_raw_data_files.checkpoints;

In [0]:
bronze_raw_df = (
    spark.readStream
        .format("cloudFiles")
        .option("cloudFiles.format", "json")
        .option(
            "cloudFiles.schemaLocation",
            "/Volumes/angad_kumar91/fraud_detection_raw_data_files/schema/bronze"
        )
        .load(
            "/Volumes/angad_kumar91/fraud_detection_raw_data_files/stream_input/"
        )
)


In [0]:
from pyspark.sql.functions import col
bronze_df = (
    bronze_raw_df
        .withColumn("_ingestion_timestamp", current_timestamp())
        .withColumn("_source_file", col("_metadata.file_path"))
        .withColumn("_file_size", col("_metadata.file_size"))
        .withColumn("_file_mod_time", col("_metadata.file_modification_time"))
)

In [0]:
(
    bronze_df.writeStream
        .format("delta")
        .outputMode("append")
        .option(
            "checkpointLocation",
            "/Volumes/angad_kumar91/fraud_detection_raw_data_files/checkpoints/bronze/"
        )
        .trigger(availableNow=True)
        .table(
            "angad_kumar91.fraud_detection_bronzelayer.Stream_bronze_data"
        )
)


In [0]:
spark.sql("""
SELECT
  COUNT(*) AS total_rows,
  MIN(_ingestion_timestamp) AS first_ingested,
  MAX(_ingestion_timestamp) AS last_ingested
FROM angad_kumar91.fraud_detection_bronzelayer.stream_bronze_data
""").show()


In [0]:
silver_raw_df = spark.readStream.table(
    "angad_kumar91.fraud_detection_bronzelayer.stream_bronze_data"
)


In [0]:
display(spark.read.table("angad_kumar91.fraud_detection_bronzelayer.stream_bronze_data").limit(20))